# Fruit Juice Selection Toy Problem D

The purpose of this example is to illustrate the use of the technology selection component of the optimisation problem contained within the `mola` package.

## Problem Statement

A UK citrus fruit juice producer wishes to produce 100kg of fruit juice consisting of either oranges from the US, mandarins from South Africa or apples from Italy. In order to produce the juice, they need to transport the fruit from each country to their UK processing plant. Freight transport is provided by sea container ship from the US, lorry EURO1 >32 tonnes from South Africa and diesel freight train from Italy. 

The juice producer would like to choose the type of fruit juice that minimises the environmental impact of green house gases. It takes 2kg of fruit to produce 1kg of juice for each fruit.

Assumptions
* The cost of purchasing and transporting fruit is ignored.

## Specification

In this notebook we use abstract specification v5 to solve this problem.

In [ ]:
import mola.specification5 as ms
from importlib import reload
spec = ms.ScheduleSpecification()
spec.abstract_model.AP.pprint()

# User Configuration

We need to build a new set configuration file for the fruit processes and flows for the problem.

In [ ]:
import mola.pyomoio as pyoio
selection_set_file_name = 'selection_set_data.json'
user_sets = pyoio.get_model_user_sets(spec, selection_set_file_name)
user_sets

## Sets

To configure the model we need lookup tables to select sets. These come from a database so part of the new model configuration is a link to a database.

In [ ]:
import mola.dataview as dv
import mola.dataimport as di
import pandas as pd
from pyomo.environ import *
conn = di.get_sqlite_connection()
lookup = dv.LookupTables(conn)

We show a widget that loads and saves the user configuration of sets in a tabbed interface. Existing user configuration data is loaded if there is a saved model data file.

In [ ]:
import mola.widgets as mw
lookups = {'F_m': lookup.get('flows'), 'P_m': lookup.get('processes'), 'F_t': lookup.get('flows'), 
           'P_t': lookup.get('processes'), 'F_s': lookup.get('flows'), 
           'P_s': lookup.get('processes'), 'KPI': lookup.get('KPI')}
vbox, tab = mw.get_sets(spec, lookups, selection_set_file_name)
vbox

Below, we load the sets defined from the JSON file.

In [ ]:
import json
with open(selection_set_file_name) as s:
    set_data = json.load(s)
set_data

## Parameters

The parameters are defined using the widget below.

In [ ]:
import qgrid
selection_parameters_file_name = 'selection_parameters_data.json'
param_dfr, param_dict = pyoio.get_model_user_parameters(spec, set_data, selection_parameters_file_name)
mw.get_parameters(param_dfr, selection_parameters_file_name)

Save the parameters above to the file system to persist any changes before proceeding. For reference, here is the contents of the model parameter file.

In [ ]:
with open(selection_parameters_file_name) as fp:
    params_json = json.load(fp)
params_json

# Model Build

To build the model we need to calculate the environmental impacts for each flow, process and KPI from the database. This is done as part in the `populate` method.

In [ ]:
json_files = ['selection_set_data.json', selection_parameters_file_name]
#model_instance = spec.populate(db_file, json_files, elementary_flow_ref_ids=['e1', 'e2'])
model_instance = spec.populate(json_files)
model_instance.F_m.pprint()

# Solver

We need to activate an objective or form a weighted sum of objectives. For this problem we just need the first objective of minimising the environmental impact.

In [ ]:
model_instance.obj1.activate()
model_instance.obj2.deactivate()
model_instance.obj.deactivate()

In [ ]:
opt = SolverFactory("glpk")
results = opt.solve(model_instance)
results.write()

The variable output is shown below

In [ ]:
model_instance.Flow.pprint()

In [ ]:
model_instance.Specific_Material_Transport_Flow.pprint()

In [ ]:
model_instance.Specific_Transport_Flow.pprint()

In [ ]:
model_instance.transport_constraint.pprint()

# Output

We use the `mola.output` module to prettify the output.

In [ ]:
import mola.output as mo
lookup = dv.LookupTables(conn)

In [ ]:
flows_dfr = mo.get_entity(model_instance.component('Flow'), lookup).reset_index(drop=True)
flows_dfr